## RSV Retry Mean
Took out the data from 2020-2021, 2021-2022.

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from statistics import mean
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
from statistics import mean
import math

FUTURE_PREDICTION = 4

def findmax(x):
    maximum = 0
    for number in x:
        maximum = max(x)
    return maximum

def findmean(x):
    m = 0
    m = mean(x)
    return m

def findmin(x):
    m = 0
    m = min(x)
    return m

def CreateMean(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Mean'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            mean = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Mean'])
            mean = findmean(list)
            dataframe.at[x,f'{title}_Mean'] = mean
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMax(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Max'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            maximum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Max'])
            maximum = findmax(list)
            dataframe.at[x,f'{title}_Max'] = maximum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMin(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Min'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            minimum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Min'])
            minimum = findmin(list)
            dataframe.at[x,f'{title}_Min'] = minimum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe
    
def DataPreprocess(city):
    virus_df = pd.read_csv('Data/virus.csv')
    virus_df['Date'] = pd.to_datetime(virus_df['Date'])
    virus_df['year'] = virus_df['Date'].apply(lambda date: date.year)
    virus_df['month'] = virus_df['Date'].apply(lambda date: date.month)
    virus_df_new = virus_df[['Date','month','year',f'{city}']]
    virus_df_new.rename(columns = {f'{city}' : f'Virus_Count_{city}'}, inplace = True)

    temp_df = pd.read_csv(f'Data/{city}_temp.csv')
    pressure_df = pd.read_csv(f'Data/{city}_pressure.csv')
    hours_df = pd.read_csv(f'Data/{city}_hours.csv')
    rainfall_df = pd.read_csv(f'Data/{city}_rainfall.csv')

    temp_df['Date'] = pd.to_datetime(temp_df['Date'])
    temp_df['year'] = temp_df['Date'].apply(lambda date: date.year)
    temp_df['month'] = temp_df['Date'].apply(lambda date: date.month)

    pressure_df['Date'] = pd.to_datetime(pressure_df['Date'])
    pressure_df['year'] = pressure_df['Date'].apply(lambda date: date.year)
    pressure_df['month'] = pressure_df['Date'].apply(lambda date: date.month)
    pressure_df = pressure_df.sort_values(by="Date")
    pressure_df = pressure_df.reset_index(drop=True)

    hours_df['Date'] = pd.to_datetime(hours_df['Date'])
    hours_df['year'] = hours_df['Date'].apply(lambda date: date.year)
    hours_df['month'] = hours_df['Date'].apply(lambda date: date.month)
    hours_df = hours_df.sort_values(by="Date")
    hours_df = hours_df.reset_index(drop=True)

    rainfall_df['Date'] = pd.to_datetime(rainfall_df['Date'])
    rainfall_df['year'] = rainfall_df['Date'].apply(lambda date: date.year)
    rainfall_df['month'] = rainfall_df['Date'].apply(lambda date: date.month)
    rainfall_df = rainfall_df.sort_values(by="Date")
    rainfall_df = rainfall_df.reset_index(drop=True)

    list = []
    for x in temp_df.index:
        t = temp_df.iloc[x][f'{city}']
        svp = 6.1078*10**(7.5*t/(t+237.3))
        e = pressure_df.iloc[x][f'{city}']
        h = e/svp*100
        list.append(h)
        
    humidity_df = pd.DataFrame(data=list)
    humidity_df.rename(columns = {0:f'{city}'}, inplace = True)

    mean_temp_df = CreateMean(temp_df, 'Temperature',f'{city}')
    mean_pressure_df = CreateMean(pressure_df, 'Pressure',f'{city}')
    mean_hours_df = CreateMean(hours_df, 'Hours',f'{city}')
    mean_rainfall_df = CreateMean(rainfall_df, 'Rainfall',f'{city}')
    mean_humidity_df = CreateMean(humidity_df, 'Humidity',f'{city}')

    df = pd.concat([virus_df_new,mean_temp_df,mean_pressure_df,mean_hours_df,mean_rainfall_df,mean_humidity_df], axis=1)
    df = df.loc[:,~df.columns.duplicated()]

    df['VirusCRM'] = df['Virus_Count_Tokyo'].rolling(14, center = True).mean()

    for i in df.index:
        if i == 0:
            df.at[i,'Gradient'] = 1
        else:
            if df.iloc[i-1]['VirusCRM'] == 0:
                if df.at[i,'VirusCDM'] > 0:
                    df.at[i,'Gradient'] = 1
                else:
                    df.at[i,'Gradient'] = 0
            else:
                grd = df.iloc[i]['VirusCRM']/(df.iloc[i-1]['VirusCRM'])
                df.at[i,'Gradient'] = grd
    
    for i in df.index:
        if (df.iloc[i]['Gradient'] >= 1.1) and (df.iloc[i+1]['Gradient'] >= 1.1) and (df.iloc[i+2]['Gradient'] >= 1.1):
            df.at[i,'Spreading'] = 1
        else:
            df.at[i,'Spreading'] = 0
    df = df.astype({'Spreading':'int8'})

    df_virus_count = df[f'Virus_Count_{city}']
    df.drop(columns=['VirusCRM',f'Virus_Count_{city}','year','Gradient'], axis=1, inplace=True)
    df_labels = df[['Date']]
    FUTURE_PREDICTION = 4
    for i in range(FUTURE_PREDICTION):
        df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)
    temp = df[["Date", "Spreading"]]
    for i in range(FUTURE_PREDICTION):
        target_column = f"T+{(i+1)*7}"
        df_labels = df_labels.merge(temp, 
                                how ="left",
                                left_on = (target_column),
                                right_on=("Date"),
                                suffixes = ("","_y")
        )
        df_labels.rename(columns={"Spreading":f"{target_column}_Spreading"},inplace=True)
    df_labels.drop(columns=["Date_y"],inplace=True)

    for i in range(FUTURE_PREDICTION):
        df_labels.drop(columns =[f"T+{(i+1)*7}"],inplace=True,axis=1 )

    df = df.merge(df_labels,
             how='left',
             on = ["Date"])
    df.dropna(inplace=True)

    X = df.drop(columns=['T+7_Spreading','T+14_Spreading','T+21_Spreading','T+28_Spreading'],axis=1)
    y1 = df['T+21_Spreading']
    y2 = df['T+7_Spreading']
    y3 = df['T+14_Spreading']
    y4 = df['T+28_Spreading']

    return X, y1, y2, y3, y4, df, df_virus_count

In [3]:
X, y1, y2, y3, y4, df, df_virus_count = DataPreprocess('Tokyo')

/Users/miyanishio/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/k5/3yc13bs57f7637937yjlstth0000gn/T/ipykernel_11196/250972663.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)


In [4]:
X

,Date,month,Temperature_Mean,Pressure_Mean,Hours_Mean,Rainfall_Mean,Humidity_Mean,Spreading
0,2012-09-10,9,28.085714,26.942857,6.928571,0.928571,71.007732,0
1,2012-09-17,9,28.414286,26.200000,8.257143,1.714286,67.730487,0
2,2012-09-24,9,24.685714,24.600000,3.614286,19.285714,78.780351,0
3,2012-10-01,10,23.314286,20.128571,4.928571,2.928571,69.795654,0
4,2012-10-08,10,21.942857,19.657143,3.928571,9.214286,74.175200,0
...,...,...,...,...,...,...,...,...
477,2021-11-01,11,15.328571,12.642857,5.528571,4.785714,72.719746,0
478,2021-11-08,11,16.085714,12.257143,7.271429,0.357143,66.710138,0
479,2021-11-15,11,14.771429,10.428571,7.600000,6.785714,61.393552,0
480,2021-11-22,11,13.142857,11.185714,4.957143,5.285714,74.012661,0


In [5]:
X = X.drop(columns=['Date'])

In [6]:
X = X[0:400] #until 2020 05-04
y1 = y1[0:400]
y2 = y2[0:400]
y3 = y2[0:400]
y4 = y2[0:400]

In [7]:
X

,month,Temperature_Mean,Pressure_Mean,Hours_Mean,Rainfall_Mean,Humidity_Mean,Spreading
0,9,28.085714,26.942857,6.928571,0.928571,71.007732,0
1,9,28.414286,26.200000,8.257143,1.714286,67.730487,0
2,9,24.685714,24.600000,3.614286,19.285714,78.780351,0
3,10,23.314286,20.128571,4.928571,2.928571,69.795654,0
4,10,21.942857,19.657143,3.928571,9.214286,74.175200,0
...,...,...,...,...,...,...,...
395,4,12.271429,9.014286,6.542857,4.428571,63.967539,0
396,4,11.557143,8.471429,6.914286,19.928571,63.265454,0
397,4,12.300000,9.728571,6.228571,15.285714,68.356621,0
398,4,13.657143,9.857143,6.671429,2.000000,62.859690,0


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
scaler1 = StandardScaler()
X_train, X_test = X[0:280], X[280:400]
y1_train, y1_test = y1[0:280], y1[280:400]

In [9]:
scaled_X_train = scaler1.fit_transform(X_train)
scaled_X_test = scaler1.transform(X_test)

In [10]:
from sklearn.svm import SVC

In [11]:
param_grid = {
 'C': [0.1,1, 10, 100, 1000], 
 'gamma': [1,0.1,0.01,0.001], 
 'kernel': ['rbf','linear','sigmoid']
} 

In [12]:
from sklearn.model_selection import GridSearchCV

In [13]:
grid = GridSearchCV(SVC(),param_grid,verbose=3)

In [14]:
grid.fit(scaled_X_train,y1_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.839 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.839 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.839 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.839 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.821 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.821 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.929 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=1, kernel=sigmoid;, score=0.821 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=1, kernel=sigmoid

[CV 4/5] END ......C=10, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 5/5] END ......C=10, gamma=1, kernel=linear;, score=0.750 total time=   0.0s
[CV 1/5] END .....C=10, gamma=1, kernel=sigmoid;, score=0.786 total time=   0.0s
[CV 2/5] END .....C=10, gamma=1, kernel=sigmoid;, score=0.857 total time=   0.0s
[CV 3/5] END .....C=10, gamma=1, kernel=sigmoid;, score=0.821 total time=   0.0s
[CV 4/5] END .....C=10, gamma=1, kernel=sigmoid;, score=0.821 total time=   0.0s
[CV 5/5] END .....C=10, gamma=1, kernel=sigmoid;, score=0.804 total time=   0.0s
[CV 1/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.911 total time=   0.0s
[CV 2/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.893 total time=   0.0s
[CV 3/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.911 total time=   0.0s
[CV 4/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.893 total time=   0.0s
[CV 5/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.875 total time=   0.0s
[CV 1/5] END ....C=10, gamma

[CV 1/5] END ....C=1000, gamma=1, kernel=linear;, score=0.821 total time=   0.1s
[CV 2/5] END ....C=1000, gamma=1, kernel=linear;, score=0.821 total time=   0.1s
[CV 3/5] END ....C=1000, gamma=1, kernel=linear;, score=0.893 total time=   0.2s
[CV 4/5] END ....C=1000, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 5/5] END ....C=1000, gamma=1, kernel=linear;, score=0.768 total time=   0.1s
[CV 1/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.786 total time=   0.0s
[CV 2/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.857 total time=   0.0s
[CV 3/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.804 total time=   0.0s
[CV 4/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.821 total time=   0.0s
[CV 5/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.786 total time=   0.0s
[CV 1/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.893 total time=   0.0s
[CV 2/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.929 total time=   0.0s
[CV 3/5] END .....C=1000, ga

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'sigmoid']},
             verbose=3)

In [15]:
grid.best_params_

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}

In [16]:
model_SVC = SVC(C=10, gamma=1,kernel='rbf')

In [17]:
model_SVC.fit(scaled_X_train, y1_train)

SVC(C=10, gamma=1)

In [18]:
preds = model_SVC.predict(scaled_X_test)

In [19]:
from sklearn.metrics import classification_report,confusion_matrix

In [20]:
print(classification_report(y1_test,preds))

              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95       108
         1.0       0.50      0.33      0.40        12

    accuracy                           0.90       120
   macro avg       0.71      0.65      0.67       120
weighted avg       0.89      0.90      0.89       120

